In [2]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice, getDevice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch

In [3]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder, outdataset):
   #outdir = os.path.join('output', outdataset, outfolder, 'Jacobian_Deformable')
   #outname = outdir + '/DetJac_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
   #print(outname)
   #jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 0 1"
   #subprocess.call(jacobian_command.split(" "))
    ####################
    outdir = os.path.join('output', outdataset, outfolder, 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii'
    print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))
    
def getLog10Jacobian_affine_ants(warp_img, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Log10Jacobian_Affine')
    outname = outdir + '/Log10JacobianAffine_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))
    
def getLog10Jacobian_affinedeformable_ants(aff_log10jac, def_log10jac, outfolder):
    outdir = os.path.join('output', 'GIN', outfolder, 'Log10Jacobian_AffineDeformable')
    outname = outdir + '/Log10JacobianAffineDeformable' + aff_log10jac.split('/')[-1].split('Log10JacobianAffine')[-1]
    combine_command = "ImageMath 3 " + str(outname) + " + " + str(aff_log10jac) + " " + str(def_log10jac)
    subprocess.call(combine_command.split(" "))

In [ ]:
#outfolders = ["lambda1_0.001",
#              "lambda1_0.01",
#              "lambda1_0.1",
#              "lambda1_1.0",
#              "lambda1_10",
#              "lambda1_100",
#              "lambda1_1000",
#             ]
#
#models = [
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-0.001.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-0.01.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-0.1.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-1.0.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-10.0.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-100.0.pth",
#    "lambda1_feminad/lambda1_feminad_1.0-0.0-1000.0.pth",
#         ]


#outfolders = [
#              "lambda2_0.01",
#              "lambda2_0.001",
#              "lambda2_0.1",
#              "lambda2_1.0",
#              "lambda2_10",
#              "lambda2_100",
#              "lambda2_1000",
#             ]
#

# models = [
#    "lambda2_sym_gin/lambda2_sym0.01_gin2_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym0.001_gin1_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym0.1_gin3_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym1_gin4_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym10_gin5_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym100_gin6_1.0-0.0-1.0.pth",
#    "lambda2_sym_gin/lambda2_sym1000_gin7_1.0-0.0-1.0.pth",
#    ]


#models = ["feminad_sym/sym_feminad_1.0-0.0-1.0.pth",
#         "feminad_sym/sym_feminad_1.0-0.0-8.0.pth",]
#
#outfolders = ["feminad-sym-1",
#             "feminad-sym-8",
#             ]
    
#models = [
#    "sym_ginfeminad_1.0-0.0-1.0.pth",
#]
#    
#outfolders = [
#    "gin+feminad",
#]
#
#models = [
#    "lambda2_sym0.01_gin_noaug_1.0-0.0-1.0.pth",
#]
#    
#outfolders = [
#    "lambda2_0.01_noaug",
#]

#models = [
#        "old_final/scenario2_ddf_1.0-0.0-8.0.pth",
#        "old_final/scenario3_ddf_1.0-0.0-8.0.pth",
#]
#
#outfolders = [
#    "dl-painfact",
#    "dl-painfact+op",
#]

#models = [
#    "sym_giniris_1.0-0.0-1.0.pth",
#]
#
#outfolders = [
#    "gin+iris",
#]

#outfolders = [
#    "test",    
#             ]
#
#models = [
#    "feminad3_test_1.0-0.0-1.0.pth",
#]
#
#    
#outdataset = 'Femina3'

In [7]:
outfolders = [  
    "dl-iris-sym",
]
models = [ 
    "op_lambda1_gin/lambda1_gin_1.0-0.0-1.0.pth",
]
outdataset = 'GIN'

#device = getDevice()
#for i, model in enumerate(models):  
#    outfolder = outfolders[i]
#    mris = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_affine.nii.gz")))
#    labels = sorted(glob(os.path.join('dataset3', outdataset, 'Labels', "*_affine_convert.nii.gz")))
#    warp_nearest = Warp("nearest", "zeros")
#    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
#    atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
#    
#    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
#    affine = nib.load(ants_warp[0]).affine
#    header = nib.load(ants_warp[0]).header     
#    
#    affine_atlas = nib.load(atlas_labels).affine
#    header_atlas = nib.load(atlas_labels).header 


for i, qsdqdsdsq in enumerate(outfolders):
    outfolder = outfolders[i]
    mridir = 'output/' + outdataset + '/' + outfolder + '/'
    affine_warps = sorted(glob(os.path.join('output', 'GIN', 'ANTS', 'AffineWarp', "*.nii.gz")))
    deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*_warp.nii.gz")))
    for i in range(len(deformable_warps)):  
        getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  
    #aff_log10jacs = sorted(glob(os.path.join('output', 'GIN', 'ANTS', 'Log10Jacobian_Affine', "*.nii.gz")))
    #def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
    #for i in range(len(aff_log10jacs)):
    #    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)


output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_653_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_655_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_656_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_660_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_661_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_662_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_663_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Deformable/Log10JacobianDeformable_664_N4_o_128_ras_norm_id_affine_warp.nii.gz.nii
output/GIN/dl-iris-sym/Log10Jacobian_Def

In [5]:
outfolders = [  
    "lambda1_1.0",
]
models = [ 
    "op_lambda1_gin/lambda1_gin_1.0-0.0-1.0.pth",
]
outdataset = 'GIN'

device = getDevice()
for i, model in enumerate(models):  
    outfolder = outfolders[i]
    mris = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_affine.nii.gz")))
    labels = sorted(glob(os.path.join('dataset3', outdataset, 'Labels', "*_affine_convert.nii.gz")))
    warp_nearest = Warp("nearest", "zeros")
    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
    atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
    
    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
    affine = nib.load(ants_warp[0]).affine
    header = nib.load(ants_warp[0]).header     
    
    affine_atlas = nib.load(atlas_labels).affine
    header_atlas = nib.load(atlas_labels).header 

    for i, mri in enumerate(mris):
        outname = "output/" + outdataset + "/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0]
        if "newmodel" in model:
            newmodel = True
        else:
            newmodel = False
            
        if "sym" in model:
            sym = True
            pred_images, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)
            pred_image = pred_images[0]
            predsym_image = pred_images[1]
        else:
            sym = False
            pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)

        moving_image = ddfs[0].cpu()

        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
        deformable_ddf = deformable_ddf * [-1, -1, 1]
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
        
        if outdataset == 'GIN':
            moving_labels = torch.from_numpy(nib.load(labels[i]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
            pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
            outname_label = "output/GIN/" + outfolder + "/Labels/" +  mri.split('/')[-1].split('.')[0] + "_labels.nii.gz"
            labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
            nib.save(labels_image, outname_label)
            print(outname_label)
        
        outname_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_warp.nii.gz"
        nib.save(deformable_image, outname_deformable)
        print(outname_deformable)
        
        if sym:
            deformable_sym_ddf = ddfs[2].cpu()
            deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
            deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
            deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
            deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
            outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
            nib.save(deformable_sym_image, outname_sym_deformable)
            
            if outdataset == 'GIN':
                moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
                pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
                outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mri.split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
                labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
                nib.save(labels_image, outname_label)
                print(outname_label)


#for i, model in enumerate(models):
    #outfolder = outfolders[i]
    #mridir = 'output/' + outdataset + '/' + outfolder + '/'
    #affine_warps = sorted(glob(os.path.join('output', 'GIN', 'ANTS', 'AffineWarp', "*.nii.gz")))
    #deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*_warp.nii.gz")))
    #for i in range(len(deformable_warps)):  
    #    getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  
    #aff_log10jacs = sorted(glob(os.path.join('output', 'GIN', 'ANTS', 'Log10Jacobian_Affine', "*.nii.gz")))
    #def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
    #for i in range(len(aff_log10jacs)):
    #    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)


(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/653_N4_o_128_ras_norm_id_affine_labels.nii.gz
output/GIN/lambda1_1.0/DeformableWarp/653_N4_o_128_ras_norm_id_affine_warp.nii.gz
(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/655_N4_o_128_ras_norm_id_affine_labels.nii.gz
output/GIN/lambda1_1.0/DeformableWarp/655_N4_o_128_ras_norm_id_affine_warp.nii.gz
(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/656_N4_o_128_ras_norm_id_affine_labels.nii.gz
output/GIN/lambda1_1.0/DeformableWarp/656_N4_o_128_ras_norm_id_affine_warp.nii.gz
(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/660_N4_o_128_ras_norm_id_affine_labels.nii.gz
output/GIN/lambda1_1.0/DeformableWarp/660_N4_o_128_ras_norm_id_affine_warp.nii.gz
(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/661_N4_o_128_ras_norm_id_affine_labels.nii.gz
output/GIN/lambda1_1.0/DeformableWarp/661_N4_o_128_ras_norm_id_affine_warp.nii.gz
(1, 1, 128, 128, 128)
output/GIN/lambda1_1.0/Labels/662_N4_o_128_ras_norm_id_affine_labels.nii.gz
ou

In [3]:
outfolders = [
    "test",    
             ]

models = [
    "feminad/sym_feminad3_1.0-0.0-1.0.pth",
]
    
device = getDevice()
for i, model in enumerate(models):
    outdataset = 'Femina3'
    
    outfolder = outfolders[i]
    mris = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_affine.nii.gz")))
    labels = sorted(glob(os.path.join('dataset3', outdataset, 'Mask', "*_affine.nii.gz")))
    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"
    atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))    
    warp_nearest = Warp("nearest", "zeros")
    affine = nib.load(ants_warp[0]).affine
    header = nib.load(ants_warp[0]).header 
    
    affine_atlas = nib.load(atlas_labels).affine
    header_atlas = nib.load(atlas_labels).header 
    
    #for i, mri in enumerate(mris):
    #    outname = "output/" + outdataset + "/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0]
    #    if "newmodel" in model:
    #        newmodel = True
    #    else:
    #        newmodel = False
    #        
    #    if "sym" in model:
    #        sym = True
    #        pred_images, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)
    #        pred_image = pred_images[0]
    #        predsym_image = pred_images[1]
    #    else:
    #        sym = False
    #        pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)
#
    #    moving_image = ddfs[0].cpu()
#
    #    #deformable_ddf = ddfs[1].cpu()
    #    #deformable_ddf = deformable_ddf.cpu().numpy()
    #    #deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    #    #deformable_ddf = deformable_ddf * [-1, -1, 1]
    #    #deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
    #    #outname_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_warp.nii.gz"
    #    #nib.save(deformable_image, outname_deformable)
    #    
    #    moving_labels = torch.from_numpy(nib.load(labels[i]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    #    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    #    outname_label = "output/Femina3/" + outfolder + "/Mask/" +  mri.split('/')[-1].split('.')[0] + "_mask.nii.gz"
    #    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    #    nib.save(labels_image, outname_label)
    #    print(outname_label)
    #    
    #    if sym:
    #        deformable_sym_ddf = ddfs[2].cpu()
    #        deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
    #        deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
    #        deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
    #        deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
    #        outname_sym_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
    #        nib.save(deformable_sym_image, outname_sym_deformable)
    #        
    #        #moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    #        #pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
    #        #outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mri.split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
    #        #labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
    #        #nib.save(labels_image, outname_label)
    #        #print(outname_label)
        
    mridir = 'output/' + outdataset + '/' + outfolder + '/'
    affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
    deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*_warp.nii.gz")))
    for i in range(len(deformable_warps)):
        #getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
        print(deformable_warps[i])
        getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  

/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/monai/networks/blocks/warp.py:66: UserWarning:

monai.networks.blocks.Warp: Using PyTorch native grid_sample.



(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6827_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6828_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6831_dki_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6835_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6836_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6837_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6838_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6839_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6840_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/2mois_6841_dki_M_N4_128_ras_norm_id_aff

(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6845_dki_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6846_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6847_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6848_dki_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6849_dki_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6859_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6860_wt_F_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6863_wt_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6864_wt_M_N4_128_ras_norm_id_affine_mask.nii.gz
(1, 1, 128, 128, 128)
output/Femina3/test/Mask/4mois_6874_dki_F_N4_128_ras_norm_id_aff

output/Femina3/test/DeformableWarp/2mois_6845_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_6845_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_6846_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_6846_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_6847_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_6847_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_6848_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_6848_dki_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_6849_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10J

output/Femina3/test/DeformableWarp/2mois_7458_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_7458_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_7459_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_7459_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_7460_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_7460_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_7461_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_2mois_7461_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/2mois_7462_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_D

output/Femina3/test/DeformableWarp/4mois_6864_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_6864_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_6874_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_6874_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_6878_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_6878_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7407_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_7407_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7408_wt_M_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian

output/Femina3/test/DeformableWarp/4mois_7470_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_7470_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7471_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_7471_wt_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7475_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_7475_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7476_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacobian_Deformable/Log10JacobianDeformable_4mois_7476_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz.nii.gz
output/Femina3/test/DeformableWarp/4mois_7477_dki_F_N4_128_ras_norm_id_affine_warp.nii.gz
output/Femina3/test/Log10Jacob

In [ ]:
#aff_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Affine', "*.nii.gz")))
#def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
#for i in range(len(aff_log10jacs)):
#    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)